A rotatividade de clientes é um problema crucial no setor de telecomunicações, definido como a perda de clientes para outra operadora. Prever a rotatividade antecipadamente, como "este cliente vai nos deixar nos próximos X meses?", permite aplicar políticas de marketing para reter clientes e aumentar a base. Mesmo uma redução de 1% na taxa de rotatividade pode resultar em um aumento significativo nos lucros.

A análise de churn envolve estudar "um cliente", "um produto ou serviço" e "a probabilidade de abandono". O objetivo é identificar clientes propensos a sair e tomar ações preventivas antes que eles partam. 

Os dados de rotatividade que utilizo são de uma empresa fictícia que forneceu serviços de telefone e Internet para 7.043 clientes na Califórnia no terceiro trimestre. Com esses dados, desenvolvo um modelo de aprendizado de máquina para prever quais clientes deixarão a empresa, realizando análise de dados e engenharia de recursos.

O conjunto de dados Telco Customer Churn da Kaggle foi utilizado, contendo 21 colunas e 7.043 linhas com informações como customerID, sexo, serviço telefônico e de Internet. Analiso essas colunas para identificar variáveis independentes (X) e dependentes.

Esse projeto me proporciona experiência prática em ciência de dados, incluindo a manipulação de grandes volumes de dados, aplicação de técnicas de aprendizado de máquina, e implementação de estratégias de retenção de clientes, preparando-me para atuar em diversos setores e segmentos de negócios.


### 0.0 - Imports

In [1]:
# mmanipulação
import pandas as pd
import inflection
import gc

# matematica e estatistica
import numpy as np
from scipy import stats

# visualização
import matplotlib.pyplot as plt
import seaborn as sns

gc.collect()

0

### 0.1 - Help Functions

In [10]:
def show_descriptive_statistical(df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

### 0.2 - Path Definition

In [2]:
HOME_PATH = 'C:/Users/valmi/projeto_churn/'
RAW_DATA_PATH = '/data/raw/'
INTERIM_DATA_PATH = '/data/interim/'
FIGURE_PATH = '/reports/figures/'

### 0.3- Load Data

In [3]:
base_churn_raw = pd.read_csv(HOME_PATH+RAW_DATA_PATH+'WA_Fn-UseC_-Telco-Customer-Churn.csv')
base_churn_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### 1.0 - Data Descripition

In [4]:
base_churn_raw1 = base_churn_raw.copy()
base_churn_raw1.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_churn_raw1.csv')
base_churn_raw1 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_churn_raw1.csv')


| Atributo       | Significado                                                                               |
| -------------- | ----------------------------------------------------------------------------------------- |
|customerID      | valor único que identifica o cliente														 |	
|genero          |se o cliente é homem ou mulher                                                             |
|SeniorCitizen   |se o cliente é idoso ou não (1, 0)                                                         |
|Parceiro        |se o cliente tem parceiro ou não (Sim, Não) 												 |
|Dependentes     |se o cliente possui dependentes ou não (Sim, Não)                                          |
|fidelidade      |número de meses que o cliente permaneceu na empresa                                        |
|PhoneService    |se o cliente possui serviço telefônico ou não (Sim, Não)                                   |
|MultipleLines   |se o cliente possui múltiplas linhas ou não (Sim, Não, Não serviço telefônico)             |
|InternetService |provedor de serviços de internet do cliente (DSL, Fibra óptica, Não)                       |
|OnlineSecurity  |se o cliente possui segurança online ou não (Sim, Não, Sem serviço de internet)            |
|OnlineBackup    |se o cliente possui backup online ou não (Sim, Não, Sem serviço de internet)               | 
|DeviceProtection|se o cliente possui proteção de dispositivo ou não (Sim, Não, Sem serviço de internet)     |
|TechSupport     |se o cliente tem suporte técnico ou não (Sim, Não, Sem serviço de internet)                |
|StreamingTV     |se o cliente possui streaming de TV ou não (Sim, Não, Não serviço de internet)             |
|StreamingMovies |se o cliente possui streaming de filmes ou não (Sim, Não, Sem serviço de internet)         |
|Contrato        |tipo de contrato de acordo com a duração (Mês a mês, Um ano, Dois anos)                    |
|PaperlessBilling|faturas emitidas em formato sem papel (Sim, Não)                                           |
|PaymentMethod   |forma de pagamento utilizada pelo cliente                                                  |
|MonthlyCharges  |valor cobrado pelo serviço mensalmente                                                     |  
|TotalCharges    |cobranças cumulativas de serviço durante o período de assinatura (manutenção)              |



### 1.2 - Rename Columns

In [ ]:
base_churn_raw1.rename(columns={'gender':'genero', 'SeniorCitizen':'CliIdoso','Partner':'Parceiro',
								'tenure':'Fidelidade', 'DeviceProtection':'ProtecaoDispositivo', 'StreamingMovies':'StreamFilmes', 
								'PaperlessBilling':'CobrancaPapel','PaymentMethod':'FormaPagCliente',
								'MonthlyCharges':'VlCobrancaMensal','TotalCharges':'CobrancaCumulativa'})

### 1.3 - Data Dimensions

In [9]:
num_linhas, num_colunas = base_churn_raw1.shape

print('Numero de linhas', num_linhas)
print('Numero de colunas', num_colunas)

Numero de linhas 7043
Numero de colunas 22


### 1.4 - Data Types

In [11]:
base_churn_raw1.dtypes

Unnamed: 0            int64
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object